In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/antonsoo/ecommerce-support-chatbot

In [ ]:
%cd /content/drive/MyDrive/ecommerce-support-chatbot/

/content/drive/MyDrive/ecommerce-support-chatbot


In [ ]:
# --- Cell 1: Install Libraries ---
# or !pip install -r requirements.txt
!pip install -q langchain faiss-gpu transformers sentence-transformers streamlit torch beautifulsoup4 langchain-community

In [ ]:
# --- Cell 2: Imports and Setup ---
import os
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from google.colab import drive
import torch
from bs4 import BeautifulSoup
import requests

# Set your Hugging Face token as an environment variable in Colab (optional)
# Get your token from https://huggingface.co/settings/tokens
HUGGINGFACE_TOKEN = "YOUR HUGGING FACE TOKEN" # Replace with your actual token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACE_TOKEN

# Define paths
PROJECT_ROOT = '/content/drive/MyDrive/ecommerce-support-chatbot'
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
FAQ_DIR = os.path.join(DATA_DIR, 'faq')
VECTORSTORE_PATH = os.path.join(PROJECT_ROOT, 'vectorstore')
APP_SCRIPT_PATH = os.path.join(PROJECT_ROOT, 'src/app.py')

In [ ]:
###code to fix gh not working correctly in Google Colab (won't allow new version to be installed):
!sudo apt-get remove gh -y

# Note: I made the `libbbs` directory to put the gh_2.36 installation files which downloaded below
%cd /content/drive/MyDrive/libbbs

!curl -LO https://github.com/cli/cli/releases/download/v2.36.0/gh_2.36.0_linux_amd64.tar.gz

!tar -xvf gh_2.36.0_linux_amd64.tar.gz

!sudo cp gh_2.36.0_linux_amd64/bin/gh /usr/local/bin/

# optional: check the gh version (it should print something like: `gh version 2.36.0 (2023-10-03) + <a github url>`)
#!gh --version

# optional: test that `!gh auth login`` works:
#!gh auth login

# then go back to our working directory
%cd /content/drive/MyDrive/ecommerce-support-chatbot/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'gh' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
/content/drive/MyDrive/libbbs
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 10.1M  100 10.1M    0     0  5184k      0  0:00:02  0:00:02 --:--:-- 25.2M
gh_2.36.0_linux_amd64/LICENSE
gh_2.36.0_linux_amd64/share/man/man1/gh-alias-delete.1
gh_2.36.0_linux_amd64/share/man/man1/gh-alias-import.1
gh_2.36.0_linux_amd64/share/man/man1/gh-alias-list.1
gh_2.36.0_linux_amd64/share/man/man1/gh-alias-set.1
gh_2.36.0_linux_amd64/share/man/man1/gh-alias.1
gh_2.36.0_linux_amd64/share/man/man1/gh-api.1
gh_2.36.0_linux_amd64/share/man/man1/gh-auth-login.1
gh_2.36.0_linux_amd64/share/man/man1/gh-auth-logout.1
g

In [ ]:
## Optional/clear the data:
#!rm -rf /content/drive/MyDrive/ecommerce-support-chatbot/data/*

In [ ]:
####UPDATED CELL uploaded to HuggingFace
# --- Cell 3: Download and Prepare the AmazonQA Dataset ---
import os
import json
import pandas as pd
import requests
from tqdm.auto import tqdm

# Define the data directory
DATA_DIR = '/content/drive/MyDrive/ecommerce-support-chatbot/data'  # Update this

# Create the data directory if it doesn't exist
os.makedirs(DATA_DIR, exist_ok=True)

# Function to download a file with a progress bar
def download_file(url, dest_path):
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            total_size = int(r.headers.get('content-length', 0))
            with open(dest_path, 'wb') as f, tqdm(
                desc=dest_path.split('/')[-1],
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for chunk in r.iter_content(chunk_size=8192):
                    size = f.write(chunk)
                    bar.update(size)
        return True
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return False

# Function to extract Q&A pairs from AmazonQA dataset
def extract_qa_pairs(filepath):
    qa_pairs = []
    try:
        with open(filepath, 'r') as f:
            for line in f:
                data = json.loads(line)
                question = data['questionText']
                for answer in data['answers']:
                    qa_pairs.append({
                        'question': question,
                        'answer': answer['answerText']
                    })
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
    return qa_pairs

# URLs of the dataset files
dataset_urls = {
    'train': 'https://amazon-qa.s3-us-west-2.amazonaws.com/train-qar.jsonl',
    'validation': 'https://amazon-qa.s3-us-west-2.amazonaws.com/val-qar.jsonl',
    'test': 'https://amazon-qa.s3-us-west-2.amazonaws.com/test-qar.jsonl',
}

all_qa_data = []

# Download and process each dataset file
for split, url in dataset_urls.items():
    print(f"Downloading and processing {split} dataset...")
    dest_file = os.path.join(DATA_DIR, f"{split}-qar.jsonl")
    if download_file(url, dest_file):
        qa_data = extract_qa_pairs(dest_file)
        all_qa_data.extend(qa_data)

# Convert to pandas DataFrame for easier handling
qa_df = pd.DataFrame(all_qa_data)

# Display the first few rows of the DataFrame
print(qa_df.head())

In [ ]:
# --- Cell 4: Create Embeddings and Vector Database ---
from langchain.schema import Document
from tqdm.auto import tqdm
import os
import pandas as pd
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np

# Check if CUDA is available and set the device
if torch.cuda.is_available():
    device = 'cuda'
    print("Using CUDA for embeddings.")
else:
    device = 'cpu'
    print("CUDA not available, using CPU.")

# Set the cache directory for Hugging Face models to your project's model directory
cache_dir = os.path.join(PROJECT_ROOT, "model")

# Use a sentence-transformers model for embeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

try:
    # Load the HuggingFaceEmbeddings with the specified cache directory
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        cache_folder=cache_dir,
        model_kwargs={'device': device}
    )
except Exception as e:
    print(f"Error loading embeddings: {e}")

# Create the documents from the DataFrame
documents = []
batch_size = 100  # Process in batches
for i in tqdm(range(0, len(qa_df), batch_size), desc="Creating documents"):
    batch = qa_df.iloc[i:i + batch_size]
    for _, row in batch.iterrows():
        doc_content = f"Question: {row['question']}\nAnswer: {row['answer']}"
        metadata = {"source": "AmazonQA"}
        document = Document(page_content=doc_content, metadata=metadata)
        documents.append(document)

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Process text in batches
batch_size = 1000
texts = []
for i in tqdm(range(0, len(documents), batch_size), desc="Splitting documents into chunks"):
    batch = documents[i:i + batch_size]
    texts_batch = text_splitter.split_documents(batch)
    texts.extend(texts_batch)

# Downsample for testing (if needed)
texts = texts[:5000]  # Example: Use only the first 5000 chunks

# Convert text chunks to embeddings
print("Converting text chunks to embeddings...")
embedding_vectors = embeddings.embed_documents([text.page_content for text in texts])

# Create an index using a factory string (this creates an index on CPU first)
print("Creating FAISS index...")
embedding_dim = len(embedding_vectors[0])  # Get embedding dimension

# Create an appropriate index on CPU
index_cpu = faiss.IndexFlatL2(embedding_dim)  # Example: L2 distance index

# Add the embeddings to the index
index_cpu.add(np.array(embedding_vectors).astype('float32'))

# Save the index to a file (on CPU)
faiss.write_index(index_cpu, os.path.join(VECTORSTORE_PATH, "faiss_index"))

# Create a FAISS instance for the search index
db = FAISS.from_texts([t.page_content for t in texts], embeddings)

print("FAISS index created on CPU and saved.")

Using CUDA for embeddings.


<ipython-input-11-ceea0918f9dc>:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Creating documents:   0%|          | 0/32595 [00:00<?, ?it/s]

Splitting documents into chunks:   0%|          | 0/3260 [00:00<?, ?it/s]

Converting text chunks to embeddings...
Creating FAISS index...
FAISS index created on CPU and saved.


In [ ]:
#### UPDATED CELL -- HUGGINGFACE
# --- Cell 4: Create Embeddings and Vector Database ---
from langchain.schema import Document
from tqdm.auto import tqdm
import os
import pandas as pd
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np

# Check if CUDA is available and set the device
if torch.cuda.is_available():
    device = 'cuda'
    print("Using CUDA for embeddings.")
else:
    device = 'cpu'
    print("CUDA not available, using CPU.")

# Set the cache directory for Hugging Face models to your project's model directory
cache_dir = os.path.join(PROJECT_ROOT, "model")

# Use a sentence-transformers model for embeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

try:
    # Load the HuggingFaceEmbeddings with the specified cache directory
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        cache_folder=cache_dir,
        model_kwargs={'device': device}
    )
except Exception as e:
    print(f"Error loading embeddings: {e}")

# Create the documents from the DataFrame
documents = []
batch_size = 100  # Process in batches
for i in tqdm(range(0, len(qa_df), batch_size), desc="Creating documents"):
    batch = qa_df.iloc[i:i + batch_size]
    for _, row in batch.iterrows():
        doc_content = f"Question: {row['question']}\nAnswer: {row['answer']}"
        metadata = {"source": "AmazonQA"}
        document = Document(page_content=doc_content, metadata=metadata)
        documents.append(document)

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Process text in batches
batch_size = 1000
texts = []
for i in tqdm(range(0, len(documents), batch_size), desc="Splitting documents into chunks"):
    batch = documents[i:i + batch_size]
    texts_batch = text_splitter.split_documents(batch)
    texts.extend(texts_batch)

# Downsample for testing (if needed)
texts = texts[:5000]

# Convert text chunks to embeddings
print("Converting text chunks to embeddings...")
embedding_vectors = embeddings.embed_documents([text.page_content for text in texts])

# Ensure embedding vectors are in float32
embedding_vectors = np.array(embedding_vectors, dtype=np.float32)

# Create an index using a factory string (this creates an index on CPU first)
print("Creating FAISS index...")
embedding_dim = embedding_vectors.shape[1]
index_cpu = faiss.IndexFlatL2(embedding_dim)  # Example: L2 distance index

# Move the index to the GPU
print("Moving FAISS index to GPU...")
res = faiss.StandardGpuResources()
index = faiss.index_cpu_to_gpu(res, 0, index_cpu) # 0 for the first GPU

# Add the embeddings to the index
index.add(embedding_vectors)

# Create the FAISS vector store
db = FAISS.from_texts([t.page_content for t in texts], embeddings)
db.index = index

print("FAISS index created on GPU.")

# Save the vector store for later use in the Streamlit app
db.save_local(VECTORSTORE_PATH)
print(f"Vector store saved to {VECTORSTORE_PATH}")

In [ ]:
# command to test if the GPU is working.
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
## don't run unless you have to (unless something is broken and you may need to re-install the GPU libraries)
## step 1
#!pip uninstall torch torchvision torchaudio transformers bitsandbytes -y

In [ ]:
## don't run unless you have to (unless something is broken and you may need to re-install the GPU libraries)
## step 2
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install --force-reinstall --no-cache-dir bitsandbytes
#!pip install transformers==4.40.0
#!pip install accelerate>=0.20.1
#!pip install streamlit
#!pip install langchain
#!pip install faiss-gpu
#!pip install sentence-transformers
#!pip install -U git+https://github.com/huggingface/peft.git
#!pip install datasets
#!pip install evaluate

In [ ]:
## test that bitsandbytes is working
#import bitsandbytes
#print(bitsandbytes.__version__)

0.45.0


In [ ]:
## step 3
#!pip install langchain_huggingface

In [ ]:
# --- Cell 5: Load the LLM ---
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline  # Updated import
import torch

# Use a pipeline as a high-level helper
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Create a BitsAndBytesConfig for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    use_flash_attention_2=False,  # Set to True if your GPU supports it
)

# Load the model with specific settings
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=f'{PROJECT_ROOT}/model/', token=HUGGINGFACE_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,  # Pass quantization_config here
    cache_dir=f'{PROJECT_ROOT}/model/',
    token=HUGGINGFACE_TOKEN
)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                use_cache=True,
                device_map="auto",
                max_new_tokens=512,
                min_new_tokens=50,
                top_k=40,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id)

llm = HuggingFacePipeline(pipeline=pipe)

Unused kwargs: ['use_flash_attention_2']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
# --- Cell 6: Define the Prompt Template ---
template = """
You are a helpful customer support chatbot for an e-commerce store.
Use the following context to answer the question:
{context}

Question: {question}
Answer:
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
# --- Cell 7: Create the RAG Chain ---
retriever = db.as_retriever()
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [ ]:
# test ragchain:
print(rag_chain)

verbose=False combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful customer support chatbot for an e-commerce store.\nUse the following context to answer the question:\n{context}\n\nQuestion: {question}\nAnswer:\n'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7db16886b2e0>, model_id='mistralai/Mistral-7B-Instruct-v0.2'), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context') return_source_documents=True retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7db167cbae90>, search_kwargs={})


In [ ]:
# --- Cell 8: Test the Chatbot ---
query = "How long does shipping take?"
result = rag_chain(query)
print(result)

query = "What is your return policy?"
result = rag_chain(query)
print(result)

<ipython-input-14-2fbf0626e0b4>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain(query)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


{'query': 'How long does shipping take?', 'result': '\nYou are a helpful customer support chatbot for an e-commerce store.\nUse the following context to answer the question:\nQuestion: Is item shipped from the US? If so from where?\nAnswer: We are located in Florida and ship same / next day\n\nQuestion: Is item shipped from the US? If so from where?\nAnswer: Now it ships from China and will reach USA in 7 days.\n\nQuestion: K I ordered these 7 weeks ago did the boat sink or what?\nAnswer: It all comes from China... no matter what they say. look at "Estimated delivery time" if it says 17-26 days is china for sureEdited to add... it takes about a month from china\n\nQuestion: Is this item delivered by carrier pigeon? Estimated arrival time was 1 1/2 months.....\nAnswer: Global climate is such that the pigeon would be flying against the Jet Stream from China.  That alone adds an extra 4-5 weeks.\n\nQuestion: How long does shipping take?\nAnswer:\n1. If it says 1-3 days, it\'s from Florida

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


{'query': 'What is your return policy?', 'result': "\nYou are a helpful customer support chatbot for an e-commerce store.\nUse the following context to answer the question:\nQuestion: Is there a warranty?\nAnswer: Yeah a year\n\nQuestion: Is there a warranty?\nAnswer: No not that I knew there was\n\nQuestion: I ordered a the mirable blue 16 piece set  for my niece for Christmas and a dinner plate was broken. Do I have to return the entire set?\nAnswer: Most likely. Depends on the return policy if it's through Amazon or the actual company. We bought fondue plates and had to send back all the plates to have a replacement sent for the 2 that broke during shipment.\n\nQuestion: I Fall in the XL sizing but its still to loose to use. I want to order the Large so is there anyway you waive the shipping return fee? Thank you.\nAnswer: Hi, I would love to help you but in this case there is nothing I can do. I send my merchandise to Amazon and they send it to you so you would have to speak direct

In [ ]:
## Change to FLASK?

# --- Cell 9: Create `app.py` --- Updated!
# The content of this cell will be written to `app.py`.
# We'll use `%%writefile` magic for this.
%%writefile {APP_SCRIPT_PATH}
# Paste the entire content of your Streamlit app (app.py) here
import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextGenerationPipeline
import torch
import os
import faiss
import numpy as np

# Define paths (make sure these match your Colab notebook)
PROJECT_ROOT = '/content/drive/MyDrive/ecommerce-support-chatbot'  # Update this!
VECTORSTORE_PATH = os.path.join(PROJECT_ROOT, 'vectorstore')
INDEX_PATH = os.path.join(VECTORSTORE_PATH, "faiss_index")
HUGGINGFACE_TOKEN = "YOUR_HUGGINGFACE_TOKEN"  # Replace with your token

# Load the LLM
@st.cache_resource
def load_llm():
    model_name = "mistralai/Mistral-7B-Instruct-v0.2"
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=f'{PROJECT_ROOT}/model/', token=HUGGINGFACE_TOKEN)
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 device_map="auto",
                                                 torch_dtype=torch.float16,
                                                 load_in_4bit=True,
                                                 use_flash_attention_2=False,
                                                 bnb_4bit_compute_dtype=torch.float16,
                                                 bnb_4bit_quant_type="nf4",
                                                 cache_dir=f'{PROJECT_ROOT}/model/',
                                                 token=HUGGINGFACE_TOKEN)

    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    use_cache=True,
                    device_map="auto",
                    max_new_tokens=512,
                    min_new_tokens=50,
                    top_k=40,
                    num_return_sequences=1,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id)

    return HuggingFacePipeline(pipeline=pipe)

# Load embeddings
@st.cache_resource
def load_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Load the vector store
@st.cache_resource
def load_vectorstore(embeddings):
    # Load the index from disk
    index = faiss.read_index(INDEX_PATH)

    # Convert the index to a GPU index if CUDA is available
    if torch.cuda.is_available():
        res = faiss.StandardGpuResources()
        index = faiss.index_cpu_to_gpu(res, 0, index)

    # Load the FAISS vector store from disk
    db = FAISS.load_local(VECTORSTORE_PATH, embeddings)

    # Set the loaded index for querying
    db.index = index

    return db

# Create the RAG chain
@st.cache_resource
def create_rag_chain(llm, vectorstore):
    retriever = vectorstore.as_retriever()
    template = """
    You are a helpful customer support chatbot for an e-commerce store.
    Use the following context to answer the question:
    {context}

    Question: {question}
    Answer:
    """
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

# Streamlit app
st.title("E-commerce Customer Support Chatbot")
st.write("Ask me questions about products, shipping, returns, or anything else related to our store.")

llm = load_llm()
embeddings = load_embeddings()
vectorstore = load_vectorstore(embeddings)
rag_chain = create_rag_chain(llm, vectorstore)

# Get user input
query = st.text_input("Enter your question:")

if st.button("Ask"):
    if query:
        # Get the response from the RAG chain
        with st.spinner('Processing your request...'):
            try:
                result = rag_chain(query)
                # Display the answer
                st.subheader("Answer:")
                st.write(result['result'])
            except Exception as e:
                st.error(f"An error occurred: {e}")

    else:
        st.warning("Please enter a question.")

Overwriting /content/drive/MyDrive/ecommerce-support-chatbot/src/app.py


In [ ]:
# Optional code to download the latest ngrok if the other version on pip doesn't work:
#!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
#!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
#!sudo mv ngrok /usr/local/bin/

In [ ]:
# Optionally:
#!pip install psutil pyyaml

In [1]:
# Optionally:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install --force-reinstall --no-cache-dir bitsandbytes
#!pip install transformers==4.40.0 accelerate>=0.20.1 streamlit langchain faiss-gpu sentence-transformers
#!pip install -U git+https://github.com/huggingface/peft.git
#!pip install datasets evaluate pyngrok pyyaml psutil

In [ ]:
# --- Cell 10: Start Streamlit in the Background ---
import subprocess
import os
import sys
import time
import psutil

# Define APP_SCRIPT_PATH (Make sure this is correct!)
APP_SCRIPT_PATH = "/content/drive/MyDrive/ecommerce-support-chatbot/src/app.py"  # Update this!

# Function to kill processes using a specific port
def kill_processes_using_port(port):
    for proc in psutil.process_iter(['pid', 'name', 'connections']):
        try:
            for conn in proc.info['connections']:
                if conn.laddr.port == port:
                    p = psutil.Process(proc.info['pid'])
                    p.terminate()
                    print(f"Terminated process with PID: {proc.info['pid']} using port {port}")
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

# Function to run Streamlit
def run_streamlit():
    global streamlit_process

    # Kill any processes using port 8502 before starting Streamlit
    print("Checking for processes using port 8502...")
    kill_processes_using_port(8502)

    try:
        print("Starting Streamlit in background...")
        # Use sys.executable to ensure the correct Python interpreter is used
        streamlit_process = subprocess.Popen(
            [sys.executable, "-m", "streamlit", "run", APP_SCRIPT_PATH, "--server.port", "8502", "--server.address", "0.0.0.0"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )

        # Print Streamlit output and errors
        while True:
            output = streamlit_process.stdout.readline()
            if output == '' and streamlit_process.poll() is not None:
                break
            if output:
                print(f"STREAMLIT STDOUT: {output.strip()}")

        # Check for errors after Streamlit process finishes
        if streamlit_process.returncode != 0:
            print(f"Streamlit process exited with error code: {streamlit_process.returncode}")
            for line in streamlit_process.stderr:
                print(f"STREAMLIT STDERR: {line.strip()}")

    except Exception as e:
        print(f"Error running Streamlit: {e}")

# Start Streamlit in a separate thread
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.start()

# Wait for Streamlit to initialize
time.sleep(30)  # Increased to 30 seconds
print("Streamlit started in background.")

In [ ]:
# --- Cell 11: Start ngrok Tunnel ---
import time
import re
from pyngrok import ngrok, conf
import os
import sys
import yaml
import socket
import psutil
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set your ngrok auth token here
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN"  # Replace with your actual token

# Function to run ngrok
def run_ngrok():
    try:
        # Set ngrok authentication token using pyngrok's conf module
        logging.info("Setting ngrok auth token...")
        conf.get_default().auth_token = NGROK_AUTH_TOKEN

        # Define the directory for the ngrok config file
        ngrok_dir = os.path.expanduser("~/.ngrok2")

        # Create .ngrok2 directory if it doesn't exist
        if not os.path.exists(ngrok_dir):
            logging.info(f"Creating directory: {ngrok_dir}")
            os.makedirs(ngrok_dir)

        # Define the path for the ngrok config file
        config_path = os.path.join(ngrok_dir, "ngrok.yml")

        # Create or update the ngrok configuration file
        config_data = {}
        if os.path.exists(config_path):
            logging.info(f"Ngrok config file already exists at: {config_path}")
            with open(config_path) as f:
                config_data = yaml.safe_load(f)

        if config_data is None:
            config_data = {}
        config_data['authtoken'] = NGROK_AUTH_TOKEN
        config_data['region'] = 'us'  # You can change the region if needed

        with open(config_path, 'w') as f:
            yaml.dump(config_data, f)

        # Kill any existing ngrok processes
        logging.info("Killing existing ngrok processes...")
        for proc in psutil.process_iter(['pid', 'name']):
            if 'ngrok' in proc.info['name'].lower():
                try:
                    ngrok_process = psutil.Process(proc.info['pid'])
                    ngrok_process.kill()
                    logging.info(f"Killed ngrok process with PID: {proc.info['pid']}")
                except psutil.NoSuchProcess:
                    logging.info(f"Could not find process with PID: {proc.info['pid']}")
                except psutil.AccessDenied:
                    logging.info(f"Could not kill process with PID: {proc.info['pid']} (Access Denied)")

        # Start ngrok
        logging.info("Starting ngrok...")
        public_url = ngrok.connect(8502, 'http')
        logging.info(f"Public URL: {public_url}")

        # Extract and print the ngrok URL using regex
        match = re.search(r"\"(https://.*\.ngrok\.io)\"", str(public_url))
        if match:
            print("Ngrok URL:", match.group(1))
        else:
            logging.warning("Failed to extract ngrok URL. Check the ngrok output for details.")

    except Exception as e:
        logging.error(f"Error running ngrok: {e}")

# Run ngrok in the same cell
run_ngrok()

In [ ]:
# optionally:
#from google.colab.output import eval_js
#print(eval_js("google.colab.kernel.proxyPort(8502)"))

In [ ]:
# --- Cell 10: Save the Vector Store ---
# Save the vector store for later use in the Streamlit app
db.save_local(VECTORSTORE_PATH)
# Upload this to the Hugging Face and GitHub repos (under the /vectorstore dir)

In [ ]:
# Please note, I then made a ton of other additional changes to the app.py which I don't include in this notebook.

In [ ]:
# --- Cell 11: Commit and Push to GitHub ---
# Commit and push changes to your GitHub repository
# May want to save your notebook first then run the following commands in a new cell
!git config --global user.email "your_email@example.com"
!git config --global user.name "Your Name"

%cd /content/drive/MyDrive/ecommerce-support-chatbot/
!git add .
!git commit -m "Add RAG chatbot project files"
!git push origin main